<div style="
    background-color:rgb(210, 230, 255);
    color:rgb(0, 0, 0);
    padding: 10px;
    border-radius: 10px;
    font-weight: bold;">
<h3>Analyse des 2 sources de données disponible pour Cartofriches pour évaluer leur exploitabilité dans le cadre d'un modèle d'apprentissage automatique</h3>
<ul>
    <li>Dans le cadre du projet PROFIL : https://github.com/heuzef/cartofriches</li>
    <li>Auteur : Heuzef (https://heuzef.com)</li>
    <li>Juin 2025</li>
    <li>Destiné au CEREMA</li>
</ul>
</div>

# Import des librairies

In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Chargement des données enrichies de Cartofriches, contient un problème de formatage, l'option on_bad_lines='skip' permet d'ignorer la friche 02684_2572 pose problème.
# https://cartofriches.cerema.fr/cartofriches/_w_1ba3f5f6866a4d2fa8e6625cae395be5/#tab-8707-1
df_cartofriches = pd.read_csv("data/Cartofriches_export_filtres.csv", sep=",", on_bad_lines='skip')

# Chargement des données de data.gouv.fr, la 35ème colonne possède des Dtype mixte, l'option low_memory=False permet de contourner le problème.
# https://www.data.gouv.fr/fr/datasets/sites-references-dans-cartofriches/#/resources
df_datagouvfr = pd.read_csv("data/friches-standard.csv", sep=";", low_memory=False)

print("Données enrichies de Cartofriches :")
print(f"{df_cartofriches.shape[0]} friches chargées — {df_cartofriches.shape[1]} variables")
print("-----")
print("Données d'origine de data.gouv.fr :")
print(f"{df_datagouvfr.shape[0]} friches chargées — {df_datagouvfr.shape[1]} variables")

print("-----")
print(f"Il manque {df_datagouvfr.shape[0] - df_cartofriches.shape[0]} friches dans les données enrichies de Cartofriches, ce problème a déjà été remonté auprès du CEREMA.")

Données enrichies de Cartofriches :
20367 friches chargées — 11 variables
-----
Données d'origine de data.gouv.fr :
28115 friches chargées — 50 variables
-----
Il manque 7748 friches dans les données enrichies de Cartofriches, ce problème a déjà été remonté auprès du CEREMA.


# Liste des variables et comparatifs

In [4]:
display(df_cartofriches.columns)
display(df_datagouvfr.columns)

Index(['Unnamed: 0', 'Identifiant et lien Cartofriches', 'Département',
       'Commune', 'Nom du site', 'Producteur de la donnée', 'Statut', 'Type',
       'Zonage d'urbanisme', 'Surface (en Ha)',
       'Coordonnées GPS (Longitude,Latitude)'],
      dtype='object')

Index(['site_id', 'site_nom', 'site_type', 'site_adresse', 'site_identif_date',
       'site_actu_date', 'site_url', 'site_ademe_url', 'site_securite',
       'site_occupation', 'site_statut', 'site_projet_url',
       'site_reconv_annee', 'site_reconv_type', 'activite_libelle',
       'activite_code', 'activite_fin_annee', 'comm_nom', 'comm_insee',
       'bati_type', 'bati_nombre', 'bati_surface', 'bati_pollution',
       'bati_vacance', 'bati_patrimoine', 'bati_etat', 'local_ancien_annee',
       'local_recent_annee', 'proprio_type', 'proprio_personne', 'proprio_nom',
       'sol_pollution_annee', 'sol_pollution_existe', 'sol_pollution_origine',
       'sol_pollution_commentaire', 'sol_depollution_fiche',
       'unite_fonciere_surface', 'unite_fonciere_refcad', 'urba_zone_type',
       'urba_zone_lib', 'urba_zone_formdomi', 'urba_doc_type',
       'desserte_distance', 'desserte_commentaire', 'source_nom', 'source_url',
       'source_producteur', 'source_contact', 'geompoint', 'geo

<div style="
    background-color:rgb(210, 230, 255);
    color:rgb(0, 0, 0);
    padding: 10px;
    border-radius: 10px;
    font-weight: bold;">
<p>
L’enrichissement de données de Cartofriches : https://cartofriches.cerema.fr/cartofriches/_w_f2572f9b9be04b3eb0f21829f2bfbdc3/#tab-5751-2
</p>
</div>

## Indicateurs Cartofriches non existants dans data.gouv.fr (30 indicateurs) :

Cartofriches réalise de nombreux enrichissement de données, en proposant plus de 30 indicateurs, que l’on peut regrouper de la manière suivante :

### Description de la friche :
* Type de propriétaire : Qui détient la friche ? - calculé à partir des fichiers fonciers
* Surface de la friche : Quelle est la superficie de la friche et des parcelles qui la compose ? - calculée à partir de la géométrie de la friche
* Nombre de bâtis, emprise au sol et coefficient d’emprise au sol : La friche est-elle bâtie ? - calculé à partir de la BDTopo
* Pollution du sol : Le sol est-il pollué ? - information renseignée par les observatoires locaux et enrichies par les SIS
### Description réglementaire de l’environnement de la friche :
* Zonage d’urbanisme : La friche est-elle en zone urbanisée ou urbanisable ? - calculée à partir des données du Géoportail de l’urbanisme
* Risques naturels : La friche est-elle soumise à des risques naturels ? - calculée à partir des données du Géoportail de l’urbanisme
* Risques technologiques : La friche est-elle soumise à des risques technologiques ? - calculée à partir des données du Géoportail de l’urbanisme
* Zonages environnementaux : La friche est-elle sur le périmètre d’une ZNIEFF, une réserve naturelle ou une zone Natura2000 ? - calculée à partir des données du Géoportail de l’urbanisme
* Monument historique : La friche est-elle sur un périmètre de 500m autour d’un monument historique ? - calculée à partir des données data.gouv.fr
* Zone d’accélération d’énergie renouvelable : La friche est-elle située dans une zone d’accélération d’énergie renouvelable PV au sol ? - calculée à partir des données du portail des EnR
* Trame Verte et Bleue : La friche est-elle sur une trame verte et bleue ? - calculée à partir des données du Géoportail de l’urbanisme
### Autres descriptions de l’environnement de la friche :
* Accessibilité aux transports : gare, échangeur autoroutier - calculée à partir des données data.gouv.fr
* Appartenance au centre-bourg : calculée à partir de la distance à la mairie
* Proximité aux commerces : calculée à partir des données de la Base Permanente des équipements de l’INSEE
* Distance à un poste électrique : calculée à partir des données ENEDIS

## Vérification de la présence de l'information dans le standard CNIG

- Unnamed: 0 : Index, hors sujet
- Commune : Équivalent "comm_insee", mais épargne une étape de transformation pour récupérer le nom du département.
- Nom du site : Équivalent "site_nom"
- Producteur de la donnée : Équivalent "source_producteur"
- Statut : Équivalent "site_statut"
- Type : Équivalent "site_type"
- Zonage d'urbanisme : Équivalent "urba_zone_type"
- Surface (en Ha) : Équivalent de "unite_fonciere_surface" en m² converti en Ha
- Coordonnées GPS (Longitude,Latitude) : Équivalent "geompoint" format WKT

## Bilan

À ce jour, les données enrichies de Cartofriches ne présentent aucun intérêt pour le Machine Learning. L'état des données de ces deux sources porte à croire que les données "enrichies" de Cartofriches ont pour origine la source déjà présente sur data.gouv.fr respectant le format CNIG.